## import modules first

In [8]:
import random
import csv
import pandas as pd
import mysql.connector as msql
from mysql.connector import errorcode

## this class generates random character stats and stores them in a dictionary, self.stats
### sample dictionary:
```
{'name': 'Lael Brightbuckle', 'race': 'elf', 'level': 1, 'alignment': 'neutral evil', 
'strength': 8, 'dexterity': 8, 'constitution': 14, 'intelligence': 5, 'wisdom': 13, 'charisma': 12}
```

In [9]:
class character_gen:
    def __init__(self):
        level = {'level': 1}
        self.level = level

    def get_name(self):
        """ generates character name """
        first_names = {1: 'Helja', 2: 'Ilde', 3: 'Birael', 4: 'Thea', 
                       5: 'Milo', 6: 'Lael', 7: 'Marci', 8: 'Wrenn', 
                       9: 'Lorill', 10: 'Amaun'}
        first = random.randint(1,10)
        first = first_names[first]
        last_names = {1: 'Toruun', 2: 'Manbroek', 3: 'Naldo', 4: 'Brushfellow', 
                      5: 'Highthorne', 6: 'Navarro', 7: 'Faircloak', 8: 'Garreck', 
                      9: 'Brightbuckle', 10: 'Saluzir'}
        last = random.randint(1,10)
        last = last_names[last]
        name = {'name': first + ' ' + last}
        self.name = name

    def get_race(self):
        """ generates random character race and associated traits """
        races = {1: 'dwarf', 2: 'elf', 3: 'halfling', 4: 'human', 5: 'gnome'}
        x = random.randint(1,5)
        race = {'race': races[x]}
        self.race = race
        
    def get_alignment(self):
        """ generates random alignment """
        alignment_ethics = {1: 'lawful', 2: 'neutral', 3: 'chaotic'}
        ethics = random.randint(1,3)
        ethics = alignment_ethics[ethics]
        alignment_morals = {1: 'good', 2: 'neutral', 3: 'evil'}
        morals = random.randint(1,3)
        morals = alignment_morals[morals]
        alignment = {'alignment': ethics + ' ' + morals}
        self.alignment = alignment
        
    def get_abilities(self):
        """ generates random ability scores """
        abilities = {'strength': 1, 'dexterity': 1, 'constitution': 1, 'intelligence': 1, 'wisdom': 1, 'charisma': 1}
        for key in abilities:
            n = 1
            temp = []
            while n <=4:
                x = random.randint(1, 6)
                temp.append(x)
                n += 1
            temp.sort(reverse=True)
            del temp[-1]
            num = 0
            for n in temp:
                num += n
            abilities[key] = num
        self.abilities = abilities
    
    def gen(self):
        self.get_name()
        self.get_race()
        self.get_alignment()
        self.get_abilities()
        stats = {}
        stats.update(self.name)
        stats.update(self.race)
        stats.update(self.level)
        stats.update(self.alignment)
        stats.update(self.abilities)
        self.stats = stats
    


## this method generates self.stats dictionaries using the above class
## then csv DictWriter writes self.stats dictionaries to rows in a csv file

In [10]:
char = character_gen()

with open('characters.csv', 'w', newline = '') as file:
    columns = ['name', 'race', 'level', 'alignment', 
               'strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma']
    writer = csv.DictWriter(file, fieldnames = columns)
    writer.writeheader()
    n = 1
    while n <= 22:
        char.gen()
        writer.writerow(char.stats)
        n += 1

In [12]:
df = pd.read_csv('characters.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          22 non-null     object
 1   race          22 non-null     object
 2   level         22 non-null     int64 
 3   alignment     22 non-null     object
 4   strength      22 non-null     int64 
 5   dexterity     22 non-null     int64 
 6   constitution  22 non-null     int64 
 7   intelligence  22 non-null     int64 
 8   wisdom        22 non-null     int64 
 9   charisma      22 non-null     int64 
dtypes: int64(7), object(3)
memory usage: 1.8+ KB


## create a database, 'test', and create a table, 'character_stats' in MySQL
```
CREATE DATABASE test;
USE test;
DROP TABLE IF EXISTS `character_stats`;

CREATE TABLE IF NOT EXISTS `character_stats` (
`id` INT NOT NULL AUTO_INCREMENT,
`name` VARCHAR(50) NOT NULL,
`race` VARCHAR(50) NOT NULL,
`level` INT NOT NULL,
`alignment` VARCHAR(50) NOT NULL,
`strength` VARCHAR(50) NOT NULL,
`dexterity` VARCHAR(50) NOT NULL,
`constitution` VARCHAR(50) NOT NULL,
`intelligence` VARCHAR(50) NOT NULL,
`wisdom` VARCHAR(50) NOT NULL,
`charisma` VARCHAR(50) NOT NULL,
PRIMARY KEY (id)
);
```

## connect to MySQL server and insert data from csv file

In [19]:
conn = msql.connect(user = 'root', password = 'Marshmallow22', database = 'test')
print(conn.is_connected())
cursor = conn.cursor()

add_char = ("INSERT INTO test.character_stats "
           "(id, name, race, level, alignment, strength, dexterity, constitution, intelligence, wisdom, charisma) "
           "VALUES (DEFAULT, %(name)s, %(race)s, %(level)s, %(alignment)s, %(strength)s, %(dexterity)s, %(constitution)s, %(intelligence)s, %(wisdom)s, %(charisma)s)")
with open("characters.csv", 'r') as file:
    csv_file = csv.DictReader(file)
    for row in csv_file:
        data_char = dict(row)
        cursor.execute(add_char, data_char)

conn.commit()
cursor.close()
conn.close()



True


## here is a printout of all the rows from the created file, 'characters.csv'

In [3]:
with open("characters.csv", 'r') as file:
    csv_file = csv.DictReader(file)
    for row in csv_file:
        print(dict(row))  

{'name': 'Lorill Faircloak', 'race': 'dwarf', 'level': '1', 'alignment': 'chaotic evil', 'strength': '12', 'dexterity': '9', 'constitution': '14', 'intelligence': '13', 'wisdom': '7', 'charisma': '7'}
{'name': 'Thea Brightbuckle', 'race': 'human', 'level': '1', 'alignment': 'neutral neutral', 'strength': '14', 'dexterity': '11', 'constitution': '10', 'intelligence': '9', 'wisdom': '12', 'charisma': '15'}
{'name': 'Helja Manbroek', 'race': 'halfling', 'level': '1', 'alignment': 'neutral good', 'strength': '10', 'dexterity': '14', 'constitution': '11', 'intelligence': '11', 'wisdom': '15', 'charisma': '11'}
{'name': 'Lorill Saluzir', 'race': 'elf', 'level': '1', 'alignment': 'chaotic evil', 'strength': '12', 'dexterity': '17', 'constitution': '9', 'intelligence': '13', 'wisdom': '11', 'charisma': '14'}
{'name': 'Birael Toruun', 'race': 'human', 'level': '1', 'alignment': 'chaotic neutral', 'strength': '13', 'dexterity': '10', 'constitution': '16', 'intelligence': '14', 'wisdom': '8', 'ch